
# 🌀 RSVP Field Simulator (Interactive)
This notebook simulates the evolution of scalar (Φ), vector (𝒗), and entropy (𝒮) fields as a minimal conscious substrate.

Inspired by the Relativistic Scalar Vector Plenum (RSVP) theory, it models:
- Scalar diffusion and advection,
- Vector torsion and curl,
- Entropy growth via gradient complexity.

You can interactively change parameters like diffusion rate, curl influence, and time resolution.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display


In [ ]:

def laplacian(Z):
    return (
        -4 * Z
        + np.roll(Z, 1, axis=0)
        + np.roll(Z, -1, axis=0)
        + np.roll(Z, 1, axis=1)
        + np.roll(Z, -1, axis=1)
    )


In [ ]:

def run_sim(N=32, T=100, dt=0.05, diff_phi=0.1, adv_phi=0.05, curl_gain=0.01, entropy_gain=0.02):
    Phi = np.random.rand(N, N)
    vx = np.random.randn(N, N) * 0.1
    vy = np.random.randn(N, N) * 0.1
    S = np.zeros((N, N))
    snapshots = []

    for t in range(T):
        adv_x = np.roll(Phi, -1, axis=1) - np.roll(Phi, 1, axis=1)
        adv_y = np.roll(Phi, -1, axis=0) - np.roll(Phi, 1, axis=0)
        Phi += dt * (diff_phi * laplacian(Phi) - adv_phi * (vx * adv_x + vy * adv_y))

        curl = np.roll(vy, -1, axis=1) - np.roll(vx, -1, axis=0)
        vx += dt * (0.1 * laplacian(vx) + curl_gain * curl)
        vy += dt * (0.1 * laplacian(vy) - curl_gain * curl)

        grad_phi_sq = (np.roll(Phi, -1, axis=0) - Phi)**2 + (np.roll(Phi, -1, axis=1) - Phi)**2
        S += dt * (0.05 * laplacian(S) + entropy_gain * grad_phi_sq)

        if t == T - 1:
            snapshots = (Phi.copy(), vx.copy(), vy.copy(), S.copy())
    return snapshots


In [ ]:

@interact(
    diff_phi=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1, description='Φ Diffusion'),
    adv_phi=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05, description='Φ Advection'),
    curl_gain=FloatSlider(min=0.0, max=0.1, step=0.005, value=0.01, description='Curl Gain'),
    entropy_gain=FloatSlider(min=0.0, max=0.1, step=0.005, value=0.02, description='Entropy Gain'),
    T=IntSlider(min=10, max=200, step=10, value=100, description='Time Steps')
)
def plot_fields(diff_phi, adv_phi, curl_gain, entropy_gain, T):
    phi, vx, vy, s = run_sim(T=T, diff_phi=diff_phi, adv_phi=adv_phi, curl_gain=curl_gain, entropy_gain=entropy_gain)
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(phi, cmap='plasma'); axes[0].set_title('Φ (Scalar)')
    axes[1].quiver(vx, vy, scale=5); axes[1].set_title('𝒗 (Vector Field)')
    axes[2].imshow(s, cmap='inferno'); axes[2].set_title('𝒮 (Entropy)')
    axes[3].imshow(phi + s, cmap='viridis'); axes[3].set_title('Φ + 𝒮')
    for ax in axes: ax.axis('off')
    plt.tight_layout(); plt.show()
